In [1]:
import tensorflow as tf
import argparse
import os
from config import cfg
from easydict import EasyDict
from utils.kitti_loader import iterate_data
from model.group_pointcloud import FeatureNet
import numpy as np
from model.rpn import MiddleAndRPN

In [2]:
parser=EasyDict()
parser.i = 1
parser.tag = 'Test1'
parser.single_batch_size = 2
parser.lr =0.001
parser.al =1
parser.output_path = './prediction'
parser.v=False

dataset_dir = cfg.DATA_DIR
train_dir = os.path.join(cfg.DATA_DIR, 'training')
val_dir = os.path.join(cfg.DATA_DIR, 'validation')
log_dir = os.path.join('./log', parser.tag)
save_model_dir = os.path.join('./save_model', parser.tag)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(save_model_dir, exist_ok=True)

In [3]:
batches=iterate_data(train_dir,batch_size=2)
#tag,labels,vox_feature,vox_number,vox_coordinate,rgb,raw_lidar
batch=next(batches)
voxel_feature=batch[2]
voxel_coordinate=batch[4]

In [4]:
print("{}:{}".format("tag",batch[0]))
print("{}:{}".format("labels",batch[1]))
#print("{}:{}".format("vox_feature",batch[2]))
print("{}:{}".format("vox_number",batch[3]))
#print("{}:{}".format("vox_coordinate",batch[4]))
#print("{}:{}".format("rgb",batch[5]))
print("{}:{}".format("raw_lidar",batch[6][0].shape))

tag:['000000' '000001']
labels:[list(['Pedestrian 0.00 0 -0.20 712.40 143.00 810.73 307.92 1.89 0.48 1.20 1.84 1.47 8.41 0.01\n'])
 list(['Truck 0.00 0 -1.57 599.41 156.40 629.75 189.25 2.85 2.63 12.34 0.47 1.49 69.44 -1.56\n', 'Car 0.00 0 1.85 387.63 181.54 423.81 203.12 1.67 1.87 3.69 -16.53 2.39 58.49 1.57\n', 'Cyclist 0.00 3 -1.65 676.60 163.95 688.98 193.93 1.86 0.60 2.02 4.59 1.32 45.84 -1.55\n', 'DontCare -1 -1 -10 503.89 169.71 590.61 190.13 -1 -1 -1 -1000 -1000 -1000 -10\n', 'DontCare -1 -1 -10 511.35 174.96 527.81 187.45 -1 -1 -1 -1000 -1000 -1000 -10\n', 'DontCare -1 -1 -10 532.37 176.35 542.68 185.27 -1 -1 -1 -1000 -1000 -1000 -10\n', 'DontCare -1 -1 -10 559.62 175.83 575.40 183.15 -1 -1 -1 -1000 -1000 -1000 -10\n'])]
vox_number:[[2 5 9 ... 1 1 1]]
raw_lidar:(115384, 4)


In [5]:
A=FeatureNet(voxel_feature[0],voxel_coordinate[0],training=True,batch_size=2)
output=A.outputs
output.shape

TensorShape([2, 10, 400, 352, 128])

In [6]:
from utils.utils import *
anchors = cal_anchors()
pos_equal_one, neg_equal_one, targets = cal_rpn_target(batch[1], [cfg.FEATURE_HEIGHT,cfg.FEATURE_WIDTH], anchors)
pos_equal_one[..., [0]].shape
pos_equal_one_for_reg = np.concatenate([np.tile(pos_equal_one[..., [0]], 7), np.tile(pos_equal_one[..., [1]], 7)], axis=-1)
pos_equal_one_sum = np.clip(np.sum(pos_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
neg_equal_one_sum = np.clip(np.sum(neg_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
info={"pos_equal_one":pos_equal_one,"neg_equal_one":neg_equal_one,"targets":targets,
      "pos_equal_one_for_reg":pos_equal_one_for_reg,"pos_equal_one_sum":pos_equal_one_sum,
      "neg_equal_one_sum":neg_equal_one_sum}

In [7]:
model=MiddleAndRPN(output,info)

In [9]:
print(model.pos_equal_one.shape)
model.cls_neg_loss.shape
neg_equal_one_sum

(2, 200, 176, 2)


array([[[[70400.]]],


       [[[70374.]]]])

In [16]:
class RPN3D(object):
    def __init__(self,cls="Car",single_batch_size=1,learning_rate=0.001,max_gradient_norm=5.0,
                 alpha=1.5,beta=1,training=False,avail_gpus=['0']):
        self.cls = cls
        self.single_batch_size = single_batch_size
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False, dtype=tf.float32)
        self.global_step = tf.Variable(1, trainable=False)
        self.epoch = tf.Variable(0, trainable=False)
        self.epoch_add_op = self.epoch.assign(self.epoch + 1)
        self.alpha = alpha
        self.beta = beta
        self.avail_gpus = avail_gpus
         
        boundaries = [80, 120]
        values = [ self.learning_rate, self.learning_rate * 0.1, self.learning_rate * 0.01 ]
        self.lr = tf.compat.v1.train.piecewise_constant(self.epoch, boundaries, values)

        #build graph
        self.is_train=training
        
        self.vox_feature = []
        self.vox_number = []
        self.vox_coordinate = []
        self.targets = []
        self.pos_equal_one = []
        self.pos_equal_one_sum = []
        self.pos_equal_one_for_reg = []
        self.neg_equal_one = []
        self.neg_equal_one_sum = []
        
        self.delta_output = []
        self.prob_output = []
        self.opt = tf.keras.optimizers.Adam(learning_rate=self.lr)
        self.gradient_norm = []
        self.tower_grads = []
        
    def __call__(self,a):
        print(a)
        
        for idx, dev in enumerate(self.avail_gpus):
            if idx==2:
                break
            else:
                batches = iterate_data(train_dir)
                batch = next(batches)
                tag = batch[0]
                labels = batch[1]
                voxel_feature = batch[2]
                print(voxel_feature.shape)
                vox_number = batch[3]
                voxel_coordinate = batch[4]
                rgb = batch[5]
                raw_lidar = batch[6]
                # get from labels
                pos_equal_one, neg_equal_one, targets = cal_rpn_target(labels, [cfg.FEATURE_HEIGHT,cfg.FEATURE_WIDTH], anchors)
                pos_equal_one[..., [0]].shape
                pos_equal_one_for_reg = np.concatenate([np.tile(pos_equal_one[..., [0]], 7), np.tile(pos_equal_one[..., [1]], 7)], axis=-1)
                pos_equal_one_sum = np.clip(np.sum(pos_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
                neg_equal_one_sum = np.clip(np.sum(neg_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
                info={"pos_equal_one":pos_equal_one,"neg_equal_one":neg_equal_one,"targets":targets,
                      "pos_equal_one_for_reg":pos_equal_one_for_reg,"pos_equal_one_sum":pos_equal_one_sum,
                      "neg_equal_one_sum":neg_equal_one_sum}
                # model running
                @tf.function
                feature=FeatureNet(voxel_feature[0],voxel_coordinate[0],training=True,batch_size=1)
                rpn=MiddleAndRPN(inputs=feature.outputs,info=info,alpha=self.alpha, beta=self.beta, training=self.is_train)

                # input
                self.vox_feature.append(voxel_feature)
                self.vox_number.append(vox_number)
                self.vox_coordinate.append(feature.coordinate)
                self.targets.append(targets)
                self.pos_equal_one.append(pos_equal_one)
                self.pos_equal_one_sum.append(pos_equal_one_sum)
                self.pos_equal_one_for_reg.append(pos_equal_one_for_reg)
                self.neg_equal_one.append(neg_equal_one)
                self.neg_equal_one_sum.append(neg_equal_one_sum)
                
                # output
                feature_output = feature.outputs
                delta_output = rpn.delta_output
                prob_output = rpn.prob_output
                
                # loss and grad
                self.loss = rpn.loss
                self.reg_loss = rpn.reg_loss
                self.cls_loss = rpn.cls_loss
                self.cls_pos_loss = rpn.cls_pos_loss_rec
                self.cls_neg_loss = rpn.cls_neg_loss_rec

In [17]:
model=RPN3D()
model(1)

1
(1, 10144, 35, 7)


In [37]:
i=0
# batch: (tag,labels,vox_feature,vox_number,vox_coordinate,rgb,raw_lidar)
for batch in iterate_data(train_dir):
    i+=1
    if i == 2:
        break
    else:
        pass
        #print("batch:",batch[2])
        # build model
        #input data
